In [1]:
import glob
from os.path import dirname, basename
import pandas as pd
from inpainting.load import load_dataframe

In [2]:
data_dir = '../data'
results_dirs = glob.glob('../results/tracking_and_segmentation/*')


object_stats_dfs = []
for path in glob.glob(f'{data_dir}/interim/DAVIS/ObjectStats/*/raw_object_stats.csv'):
    object_stats_df = load_dataframe(path)
    object_stats_df['object_sequence_name'] = basename(dirname(path))
    object_stats_dfs.append(object_stats_df)
object_stats_df = pd.concat(object_stats_dfs)

times_dfs = []
for results_dir in results_dirs:
    for path in glob.glob(f'{results_dir}/Benchmark/*/inference_times.csv'):
        times_df = load_dataframe(path)
        times_df['model'] = basename(results_dir)
        times_df['sequence_name'] = basename(dirname(path))
        times_dfs.append(times_df)
times_dfs = pd.concat(times_dfs)

results_dfs = []
for results_dir in results_dirs:
    for path in glob.glob(f'{results_dir}/Evaluation/*/results.csv'):
        results_df = load_dataframe(path)
        results_df['model'] = basename(results_dir)
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(object_stats_df, results_df, on=['object_sequence_name', 'object_id', 'frame_id'])
df = pd.merge(df, times_dfs, on=['model', 'sequence_name', 'frame_id'])
df

,frame_id,object_id,object_to_screen_ratio,object_sequence_name,object_coverage,background_coverage,model,sequence_name,inference_time
0,0,1,0.077911,bike-packing,0.971994,0.932807,ts_dilation_3_3,bike-packing_bike-packing_001,28.730335
1,0,1,0.077911,bike-packing,0.975127,0.924044,ts_dilation_3_5,bike-packing_bike-packing_001,31.500193
2,0,1,0.077911,bike-packing,0.976400,0.919626,ts_dilation_5_3,bike-packing_bike-packing_001,28.182783
3,0,1,0.077911,bike-packing,0.979436,0.901928,ts_dilation_5_5,bike-packing_bike-packing_001,34.999073
4,0,1,0.077911,bike-packing,0.924305,0.943687,ts_no_dilation,bike-packing_bike-packing_001,27.487904
...,...,...,...,...,...,...,...,...,...
33170,70,3,0.041061,walking,0.489038,0.870395,ts_dilation_3_3,walking_walking_003,29.301216
33171,70,3,0.041061,walking,0.566704,0.859687,ts_dilation_3_5,walking_walking_003,35.937824
33172,70,3,0.041061,walking,0.600892,0.854420,ts_dilation_5_3,walking_walking_003,25.568289
33173,70,3,0.041061,walking,0.715719,0.834840,ts_dilation_5_5,walking_walking_003,29.732224


In [3]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['fps'] = df['inference_time']
metrics['object_coverage'] = df['object_coverage']
metrics['background_coverage'] = df['background_coverage']
metrics.groupby('model').mean()

,fps,object_coverage,background_coverage
model,,,
ts_dilation_3_3,24.747599,0.785507,0.963621
ts_dilation_3_5,30.080353,0.802095,0.955164
ts_dilation_5_3,27.618629,0.808829,0.950868
ts_dilation_5_5,32.443763,0.830068,0.933415
ts_no_dilation,24.699951,0.738382,0.975418
